###Task 1

In [ ]:
# Import necessary libraries
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
from PIL import Image

print("All Necessary Libraries Imported")

All Necessary Libraries Imported


In [ ]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load ConvNeXtBase model as a feature extractor
convnext_base = models.convnext_base(weights=models.ConvNeXt_Base_Weights.IMAGENET1K_V1)
convnext_base = convnext_base.to(device)
convnext_base.eval()

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=128, out_features=512, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=512, out_features=128, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (3): Linear(

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Custom Dataset
class CustomImageDataset(Dataset):
    def __init__(self, data, transform=None):
        self.images = data['data']
        self.labels = torch.tensor(data['targets']) if 'targets' in data else None
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx] if self.labels is not None else -1
        return img, label

In [ ]:
# Feature extraction function
def extract_features(data_loader, model):
    features, labels = [], []
    with torch.no_grad():
        for images, labels_batch in data_loader:
            images = images.to(device)
            batch_features = model(images)
            batch_features = batch_features.view(batch_features.size(0), -1)  # Flatten
            features.append(batch_features.cpu())
            labels.append(labels_batch)
    return torch.cat(features), torch.cat(labels)

# Train Learning with Prototypes model
def train_lwp(features, labels):
    unique_classes = torch.unique(labels)
    class_means = {}
    for cls in unique_classes:
        class_indices = (labels == cls)
        class_means[cls.item()] = features[class_indices].mean(dim=0)
    return class_means

# Predict using Learning with Prototypes (Batch Processing)
def predict_lwp(features, class_means):
    # Convert class means to a tensor for batch processing
    mean_tensor = torch.stack(list(class_means.values()))
    mean_classes = torch.tensor(list(class_means.keys()))

    # Calculate distances between features and class means
    distances = torch.cdist(features, mean_tensor)
    closest_indices = torch.argmin(distances, dim=1)
    return mean_classes[closest_indices]

In [ ]:
# Task 1
def task_1(data_paths, heldout_paths):
    print("Starting Task 1...")
    models_task1 = []
    accuracy_matrix = []

    # Precompute held-out dataset features to avoid recomputation
    heldout_features_cache = {}
    print("Precomputing features for datasets...")
    for i, path in enumerate(heldout_paths):
        print(f"Loading and processing dataset D{i+1}...")
        heldout_data = torch.load(path)
        heldout_dataset = CustomImageDataset(heldout_data, transform=transform)
        heldout_loader = DataLoader(heldout_dataset, batch_size=32, num_workers=4, shuffle=False)
        features, labels = extract_features(heldout_loader, convnext_base)
        heldout_features_cache[i] = (features, labels)
    print("Held-out dataset preprocessing completed.")

    # Iterate through datasets
    for i in range(10):
        print(f"\nProcessing Dataset D{i+1}...")

        # Load datasets
        train_data = torch.load(data_paths[i])
        train_dataset = CustomImageDataset(train_data, transform=transform)
        train_loader = DataLoader(train_dataset, batch_size=32, num_workers=4, shuffle=False)

        if i == 0:
            print("Training initial model f1 on D1...")
            train_features, train_labels = extract_features(train_loader, convnext_base)
            class_means = train_lwp(train_features, train_labels)
            models_task1.append(class_means)
            print("Initial model f1 training completed.")
        else:
            print(f"Predicting pseudo-labels for Dataset D{i+1} using model f{i}...")
            unlabeled_data = torch.load(data_paths[i])
            unlabeled_dataset = CustomImageDataset(unlabeled_data, transform=transform)
            unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=32, num_workers=4, shuffle=False)

            unlabeled_features, _ = extract_features(unlabeled_loader, convnext_base)
            pseudo_labels = predict_lwp(unlabeled_features, models_task1[-1])

            # Update model with pseudo-labels
            print("Updating model with pseudo-labeled data...")
            class_means_update = train_lwp(unlabeled_features, pseudo_labels)
            for cls in class_means_update:
                if cls in models_task1[-1]:
                    models_task1[-1][cls] = (models_task1[-1][cls] + class_means_update[cls]) / 2
                else:
                    models_task1[-1][cls] = class_means_update[cls]
            models_task1.append(models_task1[-1])
            print(f"Model f{i+1} updated successfully.")

        # Evaluate on held-out datasets
        print(f"Evaluating model f{i+1} on datasets...")
        row_accuracies = []
        for j in range(i + 1):
            heldout_features, heldout_labels = heldout_features_cache[j]
            predictions = predict_lwp(heldout_features, models_task1[-1])
            accuracy = (predictions == heldout_labels).float().mean().item() * 100
            row_accuracies.append(accuracy)
            print(f"Evaluation on D̂{j+1}: Accuracy = {accuracy:.2f}%")
        accuracy_matrix.append(row_accuracies)
        print(f"Model f{i+1} evaluation completed. Current accuracy matrix row: {row_accuracies}")

    # Print accuracy matrix
    print("\nTask 1 completed. Accuracy Matrix:")
    print("     " + "  ".join([f"D̂{i+1}" for i in range(10)]))
    for i, row in enumerate(accuracy_matrix):
        print(f"f{i+1}: " + "  ".join([f"{acc:.2f}%" for acc in row]))

    print("\nAll tasks completed successfully!")
    return models_task1, accuracy_matrix

In [ ]:
# Update these paths to point to the actual dataset locations
data_paths = [f"/content/drive/MyDrive/dataset/part_one_dataset/train_data/{i+1}_train_data.tar.pth" for i in range(10)]
heldout_paths = [f"/content/drive/MyDrive/dataset/part_one_dataset/eval_data/{i+1}_eval_data.tar.pth" for i in range(10)]

# Run Task 1
models_task1, accuracy_matrix = task_1(data_paths, heldout_paths)

torch.save(models_task1, "/content/drive/MyDrive/models_task1.pth")

Starting Task 1...
Precomputing features for datasets...
Loading and processing dataset D1...


<ipython-input-17-04e19feee6c5>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  heldout_data = torch.load(path)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/da

Loading and processing dataset D2...
Loading and processing dataset D3...
Loading and processing dataset D4...
Loading and processing dataset D5...
Loading and processing dataset D6...
Loading and processing dataset D7...
Loading and processing dataset D8...
Loading and processing dataset D9...
Loading and processing dataset D10...
Held-out dataset preprocessing completed.

Processing Dataset D1...
Training initial model f1 on D1...


<ipython-input-17-04e19feee6c5>:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(data_paths[i])


Initial model f1 training completed.
Evaluating model f1 on datasets...
Evaluation on D̂1: Accuracy = 89.00%
Model f1 evaluation completed. Current accuracy matrix row: [88.99999856948853]

Processing Dataset D2...
Predicting pseudo-labels for Dataset D2 using model f1...


<ipython-input-17-04e19feee6c5>:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unlabeled_data = torch.load(data_paths[i])


Updating model with pseudo-labeled data...
Model f2 updated successfully.
Evaluating model f2 on datasets...
Evaluation on D̂1: Accuracy = 88.44%
Evaluation on D̂2: Accuracy = 89.76%
Model f2 evaluation completed. Current accuracy matrix row: [88.44000101089478, 89.7599995136261]

Processing Dataset D3...
Predicting pseudo-labels for Dataset D3 using model f2...
Updating model with pseudo-labeled data...
Model f3 updated successfully.
Evaluating model f3 on datasets...
Evaluation on D̂1: Accuracy = 87.84%
Evaluation on D̂2: Accuracy = 89.08%
Evaluation on D̂3: Accuracy = 87.48%
Model f3 evaluation completed. Current accuracy matrix row: [87.84000277519226, 89.07999992370605, 87.48000264167786]

Processing Dataset D4...
Predicting pseudo-labels for Dataset D4 using model f3...
Updating model with pseudo-labeled data...
Model f4 updated successfully.
Evaluating model f4 on datasets...
Evaluation on D̂1: Accuracy = 87.72%
Evaluation on D̂2: Accuracy = 89.08%
Evaluation on D̂3: Accuracy = 